# Neighborhood graphs

-----

## Setup

Here are loaded the packages used in the rest of notebook:

In [1]:
use Graph;
use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Wheel;
use Graph::Path;
use Graph::Petersen;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [4]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

{charge => {iterations => 4, strength => -30}, collision => {iterations => 4, radius => 50}, link => {distance => 30}}

------

## Undirected grid graph

In [5]:
#% js
my $g = Graph::Grid.new(4, 6);

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [6]:
my $g2 = $g.neighborhood-graph(['2_1'], d => 2)

Graph(vertexes => 11, edges => 14, directed => False)

In [7]:
#% js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

------

## Directed grid graph

In [8]:
#% js
my $g = Graph::Grid.new(3, 4, :directed);

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        directed => $g.directed,
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [9]:
my $g2 = $g.neighborhood-graph(['0_0', '1_3'], d => 1)

Graph(vertexes => 7, edges => 5, directed => True)

In [10]:
$g2.vertex-list

(0_0 0_1 0_3 1_0 1_2 1_3 2_3)

In [11]:
$g2.edges

[0_0 => 0_1 0_0 => 1_0 0_3 => 1_3 1_2 => 1_3 1_3 => 2_3]

In [12]:
#% js
$g.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => [|$g2.vertex-list, |$g2.edges],
        :$background, 
        :$title-color, 
        width => 700, 
        directed => $g.directed,
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

------

## In a Petersen graph

In [13]:
#% js
my $g = Graph::Petersen.new;

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [14]:
my $g2 = $g.neighborhood-graph(["0", "1"], d => 1)

Graph(vertexes => 7, edges => 8, directed => False)

In [15]:
#%js
$g.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => [|$g2.vertex-list, |$g2.edges],
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

-----

## Neat examples

### Pick out random neighborhoods from a grid

In [16]:
my $g = Graph::Grid.new(80, 80);

my $g2 = $g.neighborhood-graph($g.vertex-list.pick(80), d => 1)

Graph(vertexes => 390, edges => 338, directed => False)

In [17]:
my $nc = $g2.weakly-connected-components.elems;
say "numbef of connected components: $nc";
my @comps = $g2.weakly-connected-components.pick($nc/3);
my @gs = @comps.map({ $g2.subgraph($_) });
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
@highlight.elems

numbef of connected components: 68


224

In [18]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => { Salmon => @highlight[0 .. @highlight.elems div 2], Moccasin => @highlight[@highlight.elems div 2 .. *]},
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 800,
        vertex-color => 'LightSkyBlue',
        edge-color => 'RoyalBlue',
        edge-thickness => 3,
        vertex-size => 4, 
        force => {charge => {strength => -30, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [19]:
#%html
$g2.dot( 
    engine => 'fdp', 
    :!node-labels, node-width => 0.2, 
    size => '12,7!', 
    highlight => { Salmon => @highlight[0 .. @highlight.elems div 2], Moccasin => @highlight[@highlight.elems div 2 .. *]}
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0_62 -->
 
 0_62 
 
 
<!-- 0_63 -->
 
 0_63 
 
 
<!-- 0_63--0_62 -->
 
 0_63--0_62 
 
 
<!-- 0_64 -->
 
 0_64 
 
 
<!-- 0_63--0_64 -->
 
 0_63--0_64 
 
 
<!-- 1_63 -->
 
 1_63 
 
 
<!-- 0_63--1_63 -->
 
 0_63--1_63 
 
 
<!-- 0_75 -->
 
 0_75 
 
 
<!-- 1_75 -->
 
 1_75 
 
 
<!-- 0_75--1_75 -->
 
 0_75--1_75 
 
 
<!-- 10_55 -->
 
 10_55 
 
 
<!-- 9_55 -->
 
 9_55 
 
 
<!-- 10_55--9_55 -->
 
 10_55--9_55 
 
 
<!-- 10_57 -->
 
 10_57 
 
 
<!-- 10_58 -->
 
 10_58 
 
 
<!-- 10_58--10_57 -->
 
 10_58--10_57 
 
 
<!-- 10_59 -->
 
 10_59 
 
 
<!-- 10_58--10_59 -->
 
 10_58--10_59 
 
 
<!-- 11_58 -->
 
 11_58 
 
 
<!-- 10_58--11_58 -->
 
 10_58--11_58 
 
 
<!-- 9_58 -->
 
 9_58 
 
 
<!-- 10_58--9_58 -->
 
 10_58--9_58 
 
 
<!-- 11_23 -->
 
 11_23 
 
 
<!-- 12_23 -->
 
 12_23 
 
 
<!-- 11_23--12_23 -->
 
 11_23--12_23 
 
 
<!-- 12_22 -->
 
 12_22 
 
 
<!-- 12_23--12_22 -->
 
 12_23--12_22 
 
 
<!-- 12_24 -->
 
 12_24 
 
 
<!-- 12_24--12_23 -->
 
 12_24--12_23 
 
 
<!-- 12_61 -->
 
 12_61 
 
 
<!-- 13_61 -->
 
 13_61 
 
 
<!-- 12_61--13_61 -->
 
 12_61--13_61 
 
 
<!-- 13_23 -->
 
 13_23 
 
 
<!-- 13_23--12_23 -->
 
 13_23--12_23 
 
 
<!-- 13_44 -->
 
 13_44 
 
 
<!-- 14_44 -->
 
 14_44 
 
 
<!-- 13_44--14_44 -->
 
 13_44--14_44 
 
 
<!-- 13_60 -->
 
 13_60 
 
 
<!-- 13_60--13_61 -->
 
 13_60--13_61 
 
 
<!-- 13_62 -->
 
 13_62 
 
 
<!-- 13_61--13_62 -->
 
 13_61--13_62 
 
 
<!-- 14_43 -->
 
 14_43 
 
 
<!-- 14_43--14_44 -->
 
 14_43--14_44 
 
 
<!-- 14_45 -->
 
 14_45 
 
 
<!-- 14_45--14_44 -->
 
 14_45--14_44 
 
 
<!-- 14_61 -->
 
 14_61 
 
 
<!-- 14_61--13_61 -->
 
 14_61--13_61 
 
 
<!-- 15_39 -->
 
 15_39 
 
 
<!-- 16_39 -->
 
 16_39 
 
 
<!-- 15_39--16_39 -->
 
 15_39--16_39 
 
 
<!-- 15_44 -->
 
 15_44 
 
 
<!-- 15_44--14_44 -->
 
 15_44--14_44 
 
 
<!-- 16_17 -->
 
 16_17 
 
 
<!-- 16_38 -->
 
 16_38 
 
 
<!-- 16_38--16_39 -->
 
 16_38--16_39 
 
 
<!-- 16_40 -->
 
 16_40 
 
 
<!-- 16_39--16_40 -->
 
 16_39--16_40 
 
 
<!-- 16_75 -->
 
 16_75 
 
 
<!-- 17_75 -->
 
 17_75 
 
 
<!-- 16_75--17_75 -->
 
 16_75--17_75 
 
 
<!-- 17_16 -->
 
 17_16 
 
 
<!-- 17_17 -->
 
 17_17 
 
 
<!-- 17_17--16_17 -->
 
 17_17--16_17 
 
 
<!-- 17_17--17_16 -->
 
 17_17--17_16 
 
 
<!-- 17_18 -->
 
 17_18 
 
 
<!-- 17_17--17_18 -->
 
 17_17--17_18 
 
 
<!-- 18_17 -->
 
 18_17 
 
 
<!-- 17_17--18_17 -->
 
 17_17--18_17 
 
 
<!-- 17_39 -->
 
 17_39 
 
 
<!-- 17_39--16_39 -->
 
 17_39--16_39 
 
 
<!-- 17_74 -->
 
 17_74 
 
 
<!-- 17_74--17_75 -->
 
 17_74--17_75 
 
 
<!-- 17_76 -->
 
 17_76 
 
 
<!-- 17_76--17_75 -->
 
 17_76--17_75 
 
 
<!-- 17_77 -->
 
 17_77 
 
 
<!-- 17_76--17_77 -->
 
 17_76--17_77 
 
 
<!-- 18_76 -->
 
 18_76 
 
 
<!-- 17_76--18_76 -->
 
 17_76--18_76 
 
 
<!-- 18_26 -->
 
 18_26 
 
 
<!-- 19_26 -->
 
 19_26 
 
 
<!-- 18_26--19_26 -->
 
 18_26--19_26 
 
 
<!-- 18_75 -->
 
 18_75 
 
 
<!-- 18_75--17_75 -->
 
 18_75--17_75 
 
 
<!-- 18_75--18_76 -->
 
 18_75--18_76 
 
 
<!-- 18_77 -->
 
 18_77 
 
 
<!-- 18_77--17_77 -->
 
 18_77--17_77 
 
 
<!-- 18_77--18_76 -->
 
 18_77--18_76 
 
 
<!-- 18_78 -->
 
 18_78 
 
 
<!-- 18_77--18_78 -->
 
 18_77--18_78 
 
 
<!-- 19_77 -->
 
 19_77 
 
 
<!-- 18_77--19_77 -->
 
 18_77--19_77 
 
 
<!-- 19_25 -->
 
 19_25 
 
 
<!-- 19_26--19_25 -->
 
 19_26--19_25 
 
 
<!-- 19_27 -->
 
 19_27 
 
 
<!-- 19_27--19_26 -->
 
 19_27--19_26 
 
 
<!-- 1_71 -->
 
 1_71 
 
 
<!-- 2_71 -->
 
 2_71 
 
 
<!-- 1_71--2_71 -->
 
 1_71--2_71 
 
 
<!-- 1_74 -->
 
 1_74 
 
 
<!-- 1_74--1_75 -->
 
 1_74--1_75 
 
 
<!-- 1_76 -->
 
 1_76 
 
 
<!-- 1_75--1_76 -->
 
 1_75--1_76 
 
 
<!-- 20_26 -->
 
 20_26 
 
 
<!-- 20_26--19_26 -->
 
 20_26--19_26 
 
 
<!-- 23_4 -->
 
 23_4 
 
 
<!-- 24_3 -->
 
 24_3 
 
 
<!-- 24_4 -->
 
 24_4 
 
 
<!-- 24_3--24_4 -->
 
 24_3--24_4 
 
 
<!-- 24_4--23_4 -->


In [20]:
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });
%vertex-coordinates.elems

6400

In [21]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        vertex-coordinates => ($g2.vertex-list Z=> %vertex-coordinates{$g2.vertex-list}).Hash,
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 800,
        edge-thickness => 3,
        vertex-size => 0, 
    )

### Subtract random neighborhoods from a grid

In [22]:
my $g = Graph::Grid.new(60, 30);
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });

my $vset = $g.vertex-list (-) $g.neighborhood-graph($g.vertex-list.pick(60), d => 1).vertex-list;

my $g2 = $g.subgraph($vset.keys)

Graph(vertexes => 1740, edges => 3260, directed => False)

In [23]:
my $g3 = $g.neighborhood-graph($g.vertex-list.pick(60), d => 2);
my $nc = $g3.weakly-connected-components.elems;
my @comps = $g3.weakly-connected-components;
my @gs = @comps.map({ $g3.subgraph($_) });
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
$nc

20

In [24]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :%vertex-coordinates,
        highlight => {Ivory => @highlight},
        :$background, 
        title-color => 'none', 
        width => 1000, 
        height => 500,
        edge-thickness => 4,
        edge-color => 'Gray',
        vertex-size => 0,
        vertex-color => 'Ivory',
        force => {charge => {strength => -30, iterations => 2}, y => {strength => 0.2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
    )

In [43]:
#%html
$g2.vertex-coordinates = %vertex-coordinates;
$g2.dot( 
    highlight => {Ivory => @highlight},
    :!node-labels, 
    node-width => 0,
    node-fill-color => 'Gray', 
    graph-size => (10, 20),
    edge-color => 'Gray',
    edge-width => 18,
    engine => 'neato', 
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 10_21 -->
 
 10_21 
 
 
<!-- 11_21 -->
 
 11_21 
 
 
<!-- 10_21--11_21 -->
 
 10_21--11_21 
 
 
<!-- 36_4 -->
 
 36_4 
 
 
<!-- 36_3 -->
 
 36_3 
 
 
<!-- 36_4--36_3 -->
 
 36_4--36_3 
 
 
<!-- 36_5 -->
 
 36_5 
 
 
<!-- 36_4--36_5 -->
 
 36_4--36_5 
 
 
<!-- 37_4 -->
 
 37_4 
 
 
<!-- 36_4--37_4 -->
 
 36_4--37_4 
 
 
<!-- 34_25 -->
 
 34_25 
 
 
<!-- 33_25 -->
 
 33_25 
 
 
<!-- 34_25--33_25 -->
 
 34_25--33_25 
 
 
<!-- 34_26 -->
 
 34_26 
 
 
<!-- 34_25--34_26 -->
 
 34_25--34_26 
 
 
<!-- 38_1 -->
 
 38_1 
 
 
<!-- 38_2 -->
 
 38_2 
 
 
<!-- 38_1--38_2 -->
 
 38_1--38_2 
 
 
<!-- 37_1 -->
 
 37_1 
 
 
<!-- 38_1--37_1 -->
 
 38_1--37_1 
 
 
<!-- 56_15 -->
 
 56_15 
 
 
<!-- 55_15 -->
 
 55_15 
 
 
<!-- 56_15--55_15 -->
 
 56_15--55_15 
 
 
<!-- 56_16 -->
 
 56_16 
 
 
<!-- 56_15--56_16 -->
 
 56_15--56_16 
 
 
<!-- 56_14 -->
 
 56_14 
 
 
<!-- 56_15--56_14 -->
 
 56_15--56_14 
 
 
<!-- 57_15 -->
 
 57_15 
 
 
<!-- 56_15--57_15 -->
 
 56_15--57_15 
 
 
<!-- 46_5 -->
 
 46_5 
 
 
<!-- 43_8 -->
 
 43_8 
 
 
<!-- 1_0 -->
 
 1_0 
 
 
<!-- 0_0 -->
 
 0_0 
 
 
<!-- 1_0--0_0 -->
 
 1_0--0_0 
 
 
<!-- 35_17 -->
 
 35_17 
 
 
<!-- 24_4 -->
 
 24_4 
 
 
<!-- 8_21 -->
 
 8_21 
 
 
<!-- 9_21 -->
 
 9_21 
 
 
<!-- 8_21--9_21 -->
 
 8_21--9_21 
 
 
<!-- 7_21 -->
 
 7_21 
 
 
<!-- 8_21--7_21 -->
 
 8_21--7_21 
 
 
<!-- 8_20 -->
 
 8_20 
 
 
<!-- 8_21--8_20 -->
 
 8_21--8_20 
 
 
<!-- 8_22 -->
 
 8_22 
 
 
<!-- 8_21--8_22 -->
 
 8_21--8_22 
 
 
<!-- 30_10 -->
 
 30_10 
 
 
<!-- 29_10 -->
 
 29_10 
 
 
<!-- 30_10--29_10 -->
 
 30_10--29_10 
 
 
<!-- 58_28 -->
 
 58_28 
 
 
<!-- 58_29 -->
 
 58_29 
 
 
<!-- 58_28--58_29 -->
 
 58_28--58_29 
 
 
<!-- 58_27 -->
 
 58_27 
 
 
<!-- 58_28--58_27 -->
 
 58_28--58_27 
 
 
<!-- 59_28 -->
 
 59_28 
 
 
<!-- 58_28--59_28 -->
 
 58_28--59_28 
 
 
<!-- 42_19 -->
 
 42_19 
 
 
<!-- 42_18 -->
 
 42_18 
 
 
<!-- 42_19--42_18 -->
 
 42_19--42_18 
 
 
<!-- 42_20 -->
 
 42_20 
 
 
<!-- 42_19--42_20 -->
 
 42_19--42_20 
 
 
<!-- 41_19 -->
 
 41_19 
 
 
<!-- 42_19--41_19 -->
 
 42_19--41_19 
 
 
<!-- 5_6 -->
 
 5_6 
 
 
<!-- 4_6 -->
 
 4_6 
 
 
<!-- 5_6--4_6 -->
 
 5_6--4_6 
 
 
<!-- 5_7 -->
 
 5_7 
 
 
<!-- 5_6--5_7 -->
 
 5_6--5_7 
 
 
<!-- 9_28 -->
 
 9_28 
 
 
<!-- 9_27 -->
 
 9_27 
 
 
<!-- 9_28--9_27 -->
 
 9_28--9_27 
 
 
<!-- 8_28 -->
 
 8_28 
 
 
<!-- 9_28--8_28 -->
 
 9_28--8_28 
 
 
<!-- 10_28 -->
 
 10_28 
 
 
<!-- 9_28--10_28 -->
 
 9_28--10_28 
 
 
<!-- 9_29 -->
 
 9_29 
 
 
<!-- 9_28--9_29 -->
 
 9_28--9_29 
 
 
<!-- 40_10 -->
 
 40_10 
 
 
<!-- 39_10 -->
 
 39_10 
 
 
<!-- 40_10--39_10 -->
 
 40_10--39_10 
 
 
<!-- 13_4 -->
 
 13_4 
 
 
<!-- 13_3 -->
 
 13_3 
 
 
<!-- 13_4--13_3 -->
 
 13_4--13_3 
 
 
<!-- 14_4 -->
 
 14_4 
 
 
<!-- 13_4--14_4 -->
 
 13_4--14_4 
 
 
<!-- 12_4 -->
 
 12_4 
 
 
<!-- 13_4--12_4 -->
 
 13_4--12_4 
 
 
<!-- 13_5 -->
 
 13_5 
 
 
<!-- 13_4--13_5 -->
 
 13_4--13_5 
 
 
<!-- 23_24 -->
 
 23_24 
 
 
<!-- 19_9 -->
 
 19_9 
 
 
<!-- 19_8 -->
 
 19_8 
 
 
<!-- 19_9--19_8 -->
 
 19_9--19_8 
 
 
<!-- 19_10 -->
 
 19_10 
 
 
<!-- 19_9--19_10 -->
 
 19_9--19_10 
 
 
<!-- 20_9 -->
 
 20_9 
 
 
<!-- 19_9--20_9 -->
 
 19_9--20_9 
 
 
<!-- 18_9 -->
 
 18_9 
 
 
<!-- 19_9--18_9 -->
 
 19_9--18_9 
 
 
<!-- 42_27 -->
 
 42_27 
 
 
<!-- 41_27 -->
 
 41_27 
 
 
<!-- 42_27--41_27 -->
 
 42_27--41_27 
 
 
<!-- 42_26 -->
 
 42_26 
 
 
<!-- 42_27--42_26 -->
 
 42_27--42_26 
 
 
<!-- 42_28 -->
 
 42_28 
 
 
<!-- 42_27--42_28 -->
 
 42_27--42_28 
 
 
<!-- 32_5 -->
 
 32_5 
 
 
<!-- 32_4 -->
 
 32_4 
 
 
<!-- 32_5--32_4 -->
 
 32_5--32_4 
 
 
<!-- 58_25 -->
 
 58_25 
 
 
<!-- 59_25 -->
 
 59_25 
 
 
<!-- 58_25--59_25 -->
 
 58_25--59_25 
 
 
<!-- 58_9 -->
 
 58_9 
 
 
<!-- 58_10 -->
 
 58_10 
 
 
<!-- 58_9--58_10 -->
 
 58_9--58_10 
 
 
<!-- 59_9 -->
